## 🔧 Setup & Installation

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except:
    IN_COLAB = False
    print("⚠️  Not in Colab - some features may not work")

# Mount Google Drive (if in Colab)
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set working directory
    import os
    PROJECT_DIR = '/content/drive/MyDrive/quantum-forecaster'
    os.makedirs(PROJECT_DIR, exist_ok=True)
    os.chdir(PROJECT_DIR)
    print(f"📁 Working directory: {PROJECT_DIR}")
else:
    PROJECT_DIR = './quantum-forecaster'

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected - training will be slower")

In [ ]:
# Install required packages
!pip install -q torch torchvision torchaudio
!pip install -q yfinance pandas numpy scikit-learn matplotlib seaborn
!pip install -q pandas-ta

# Optional: Quantum computing libraries (for advanced features)
# !pip install -q pennylane qiskit

print("✅ All packages installed")

In [ ]:
# Download model files from your repository
# If you uploaded to Google Drive, copy files to working directory

if IN_COLAB:
    # Upload the three Python files:
    # 1. quantum_forecaster_14day.py
    # 2. feature_engineering.py
    # 3. training_pipeline.py
    
    from google.colab import files
    print("📤 Upload model files (or copy from Drive)")
    # uploaded = files.upload()
    
    # Alternative: Copy from your Drive folder
    # !cp /content/drive/MyDrive/your-folder/*.py .
    
print("✅ Model files ready")

## 📊 Define Portfolio Universe

In [ ]:
# 20-ticker institutional portfolio (optimized for ML)
PORTFOLIO = [
    # Tier 1: Mega-Cap AI Leaders (70-76% accuracy expected)
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'AVGO',
    
    # Tier 2: AI/Cloud Growth (62-70% accuracy)
    'AMD', 'MRVL', 'CRM', 'DDOG', 'PLTR', 'COIN',
    
    # Tier 3: Diversified Large-Cap (58-65% accuracy)
    'JPM', 'XOM',
    
    # Your holdings
    'MU', 'IONQ'
]

# For quick testing, start with smaller subset
TEST_PORTFOLIO = ['NVDA', 'AAPL', 'MSFT', 'TSLA', 'AMD']

print(f"📊 Full Portfolio: {len(PORTFOLIO)} tickers")
print(f"🧪 Test Portfolio: {len(TEST_PORTFOLIO)} tickers")
print("\nRecommendation: Train on TEST_PORTFOLIO first (30-60 min on T4)")
print("                Then scale to full PORTFOLIO (2-3 hours)")

## 🏗️ Initialize Model & Configuration

In [ ]:
from quantum_forecaster_14day import QuantumForecastConfig, QuantumForecaster14Day
from training_pipeline import QuantumForecasterTrainer
from feature_engineering import QuantumFeatureEngineer

# Configuration
config = QuantumForecastConfig(
    # Model architecture
    d_model=256,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=4,
    
    # Quantum parameters
    n_qubits=8,
    n_quantum_layers=4,
    
    # Training
    batch_size=32,  # Reduce to 16 if GPU memory issues
    learning_rate=1e-4,
    num_epochs=50,  # Increase to 100 for production
    warmup_epochs=5,
    
    # Forecasting
    sequence_length=60,  # 60 days of history
    prediction_horizon=14,  # 14-day forecast
)

print("✅ Configuration created")
print(f"   - Model dimension: {config.d_model}")
print(f"   - Quantum qubits: {config.n_qubits}")
print(f"   - Prediction horizon: {config.prediction_horizon} days")
print(f"   - Batch size: {config.batch_size}")

## 🔄 Data Preparation & Feature Engineering

In [ ]:
# Initialize trainer
trainer = QuantumForecasterTrainer(config, device='cuda')

# Prepare data (choose TEST_PORTFOLIO for faster iteration)
USE_TEST_PORTFOLIO = True  # Set to False for full training

tickers = TEST_PORTFOLIO if USE_TEST_PORTFOLIO else PORTFOLIO

print(f"🔄 Preparing data for {len(tickers)} tickers...")
print("   This may take 5-15 minutes depending on network speed\n")

train_loader, val_loader, test_loader = trainer.prepare_data(
    tickers=tickers,
    test_size=0.2,
    val_size=0.1
)

print("\n✅ Data preparation complete!")

## 🚀 Train Model

In [ ]:
# Train the model
print("🚀 Starting training...\n")
print("⏱️  Estimated time:")
print("   - Test portfolio (5 tickers): 30-60 minutes on T4")
print("   - Full portfolio (20 tickers): 2-3 hours on T4\n")

history = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=config.num_epochs,
    patience=15
)

print("\n✅ Training complete!")

## 📊 Evaluate Performance

In [ ]:
# Evaluate on test set
results = trainer.evaluate(test_loader)

# Display metrics
print("\n" + "="*70)
print("📈 TEST SET PERFORMANCE")
print("="*70)
print(f"\n📊 Forecast Accuracy:")
print(f"   - MAE (Mean Absolute Error): {results['mae']:.4f}")
print(f"   - RMSE (Root Mean Squared Error): {results['rmse']:.4f}")
print(f"   - Directional Accuracy: {results['directional_accuracy']:.2%}")

print(f"\n📉 Uncertainty Estimation:")
print(f"   - Mean Uncertainty: {results['mean_uncertainty']:.4f}")

# Target benchmarks
print(f"\n🎯 Target Benchmarks:")
print(f"   - Directional Accuracy: 74-82%")
print(f"   - MAE: < 0.02 (2%)")

if results['directional_accuracy'] >= 0.74:
    print("\n✅ EXCELLENT: Directional accuracy meets institutional target!")
elif results['directional_accuracy'] >= 0.65:
    print("\n⚠️  GOOD: Directional accuracy is promising but below target")
else:
    print("\n❌ NEEDS IMPROVEMENT: Consider retraining with more data/epochs")

print("="*70)

## 📈 Visualize Results

In [ ]:
# Plot training history
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
trainer.plot_training_history(f'{PROJECT_DIR}/training_history.png')

In [ ]:
# Prediction vs Actual scatter plot
import numpy as np

predictions = results['predictions'][:, 0]  # First day of 14-day forecast
targets = results['targets'][:, 0]
uncertainties = results['uncertainties'][:, 0]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(targets, predictions, alpha=0.5, s=20)
axes[0].plot([-0.2, 0.2], [-0.2, 0.2], 'r--', label='Perfect prediction')
axes[0].set_xlabel('Actual 14-Day Return')
axes[0].set_ylabel('Predicted 14-Day Return')
axes[0].set_title('Predictions vs Actuals')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Uncertainty distribution
axes[1].hist(uncertainties, bins=50, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Prediction Uncertainty')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Uncertainty Distribution')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{PROJECT_DIR}/predictions_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 Visualizations saved to Drive")

## 💾 Save Model & Artifacts

In [ ]:
# Save trained model
import os

model_dir = f'{PROJECT_DIR}/models/quantum_forecaster_v1'
trainer.save_model(model_dir)

# Save results
import json

results_to_save = {
    'mae': float(results['mae']),
    'rmse': float(results['rmse']),
    'directional_accuracy': float(results['directional_accuracy']),
    'mean_uncertainty': float(results['mean_uncertainty']),
    'tickers': tickers,
    'config': config.__dict__
}

with open(f'{model_dir}/results.json', 'w') as f:
    json.dump(results_to_save, f, indent=2)

print(f"✅ Model and results saved to: {model_dir}")
print("\nYou can now use this model for:")
print("  1. Real-time 14-day forecasting")
print("  2. Trading strategy backtesting")
print("  3. Portfolio optimization")
print("  4. Risk management")

## 🔮 Make Predictions on New Data

In [ ]:
# Function to forecast any ticker
def forecast_ticker(ticker, model, scaler, config):
    """
    Generate 14-day forecast for a ticker
    """
    from feature_engineering import QuantumFeatureEngineer
    import torch
    
    print(f"🔮 Forecasting {ticker} for next 14 days...")
    
    # Get latest data
    engineer = QuantumFeatureEngineer()
    features = engineer.engineer_all_features(ticker, period='1y')
    X_price, X_micro, X_alt, X_sent, y = engineer.prepare_model_inputs(
        features,
        sequence_length=config.sequence_length,
        prediction_horizon=config.prediction_horizon
    )
    
    # Use most recent sequence
    X_price_latest = X_price[-1:]
    X_micro_latest = X_micro[-1:]
    X_alt_latest = X_alt[-1:]
    X_sent_latest = X_sent[-1:]
    
    # Scale features
    X_price_scaled, X_micro_scaled, X_alt_scaled, X_sent_scaled = scaler.transform(
        X_price_latest, X_micro_latest, X_alt_latest, X_sent_latest
    )
    
    # Convert to tensors
    X_price_t = torch.FloatTensor(X_price_scaled).to(model.device)
    X_micro_t = torch.FloatTensor(X_micro_scaled).to(model.device)
    X_alt_t = torch.FloatTensor(X_alt_scaled).to(model.device)
    X_sent_t = torch.FloatTensor(X_sent_scaled).to(model.device)
    
    # Generate forecast
    model.model.eval()
    with torch.no_grad():
        output = model.model(X_price_t, X_micro_t, X_alt_t, X_sent_t)
    
    # Extract predictions
    median_forecast = output['median_forecast'].cpu().numpy()[0]
    uncertainty = output['uncertainty'].cpu().numpy()[0]
    
    # Get quantile predictions
    quantiles = {}
    for q_name, q_pred in output['quantile_predictions'].items():
        quantiles[q_name] = q_pred.cpu().numpy()[0]
    
    # Current price
    current_price = features['all_features']['Close'].iloc[-1]
    
    print(f"\n📊 14-Day Forecast for {ticker}:")
    print(f"   Current Price: ${current_price:.2f}")
    print(f"   Expected Return: {median_forecast[0]*100:.2f}%")
    print(f"   Uncertainty: ±{uncertainty[0]*100:.2f}%")
    print(f"\n   Confidence Intervals:")
    print(f"   - 10th percentile: {quantiles['q10'][0]*100:+.2f}%")
    print(f"   - 25th percentile: {quantiles['q25'][0]*100:+.2f}%")
    print(f"   - 50th percentile (median): {quantiles['q50'][0]*100:+.2f}%")
    print(f"   - 75th percentile: {quantiles['q75'][0]*100:+.2f}%")
    print(f"   - 90th percentile: {quantiles['q90'][0]*100:+.2f}%")
    
    # Trading signal
    if median_forecast[0] > 0.02 and uncertainty[0] < 0.05:
        signal = "🟢 STRONG BUY"
    elif median_forecast[0] > 0:
        signal = "🟡 BUY"
    elif median_forecast[0] < -0.02 and uncertainty[0] < 0.05:
        signal = "🔴 STRONG SELL"
    elif median_forecast[0] < 0:
        signal = "🟠 SELL"
    else:
        signal = "⚪ HOLD"
    
    print(f"\n   Trading Signal: {signal}")
    
    return {
        'ticker': ticker,
        'current_price': current_price,
        'median_forecast': median_forecast[0],
        'uncertainty': uncertainty[0],
        'quantiles': quantiles,
        'signal': signal
    }

# Example: Forecast NVDA
nvda_forecast = forecast_ticker('NVDA', trainer, trainer.scaler, config)

In [ ]:
# Forecast entire portfolio
print("🔮 Generating forecasts for entire portfolio...\n")

portfolio_forecasts = []
for ticker in tickers:
    try:
        forecast = forecast_ticker(ticker, trainer, trainer.scaler, config)
        portfolio_forecasts.append(forecast)
        print("\n" + "-"*70 + "\n")
    except Exception as e:
        print(f"⚠️  Error forecasting {ticker}: {str(e)}\n")

# Create summary DataFrame
import pandas as pd

summary_df = pd.DataFrame([
    {
        'Ticker': f['ticker'],
        'Current Price': f['current_price'],
        '14-Day Return': f'{f["median_forecast"]*100:.2f}%',
        'Uncertainty': f'{f["uncertainty"]*100:.2f}%',
        'Signal': f['signal']
    }
    for f in portfolio_forecasts
])

print("\n📊 PORTFOLIO FORECAST SUMMARY")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

# Save summary
summary_df.to_csv(f'{PROJECT_DIR}/portfolio_forecast.csv', index=False)
print(f"\n💾 Portfolio forecast saved to {PROJECT_DIR}/portfolio_forecast.csv")

## 📚 Next Steps

### 🎯 Production Deployment
1. **Scale to Full Portfolio**: Train on all 20 tickers for maximum diversification
2. **Increase Epochs**: Train for 100-200 epochs for production-grade accuracy
3. **Hyperparameter Tuning**: Optimize d_model, nhead, quantum layers
4. **Walk-Forward Validation**: Implement rolling window retraining

### 🔄 Continuous Improvement
1. **Alternative Data Integration**: Add real satellite/credit card data sources
2. **Sentiment Enhancement**: Integrate Twitter/Reddit/News APIs
3. **Ensemble Methods**: Combine multiple model variants
4. **Regime Detection**: Add market regime classification

### 💰 Trading Integration
1. **Alpaca API**: Connect for paper/live trading
2. **Position Sizing**: Implement Kelly criterion
3. **Risk Management**: Stop-loss, portfolio limits
4. **Performance Tracking**: Sharpe, Sortino, Max Drawdown

### 📖 Resources
- **Research Papers**: Temporal Fusion Transformers, Quantum ML Finance
- **Data Sources**: Polygon.io, IEX Cloud, Finnhub
- **Community**: r/algotrading, Quantopian forums

---

## ⚠️ Risk Disclaimer
This is an educational/research implementation. Past performance does not guarantee future results. Always:
- Start with paper trading
- Use proper risk management
- Never invest more than you can afford to lose
- Consult a financial advisor before live trading

---

**Built with ❤️ by AI Trading Research Team**